Using <b>psycopg2</b> python library to <i>interact with PostgreSQL<i> and <b>pandas</b> for <i>data manipulation</i>.
Importing sql queries from <b>sql_queries</b> python file.
boto3 for connecting with AWS and using Secrets Manager Service.
json to use the loads function

In [1]:
import psycopg2
import pandas as pd
import boto3
import json
from sql_queries import *

Connect to the <b>football db</b>

In [2]:
# Function to connect to the database 
def connect_database():
        try: 
            client = boto3.client('secretsmanager',region_name='us-east-1')
            response = client.get_secret_value(
                SecretId='dev/postgress',
            )
    
            respDict=json.loads(response['SecretString'])
            # connect to the default database 
            conn=psycopg2.connect(host=respDict["host"],
                                  dbname="football_db",
                                  user=respDict["username"],
                                  password=respDict["password"])    
            cur=conn.cursor()
        except psycopg2.Error as e:
            raise e

        return conn,cur

In [3]:
# Function to execute the insert query 
def insert_data(conn,cur,file,insert_query):
    for i,row in file.iterrows():
        try:
            cur.execute(insert_query,list(row))
            conn.commit()
        except psycopg2.Error as e:
            print("Error:Inserting records into table")
            print(e)
            conn.rollback()

In [4]:
#calling the function to connect to the db
conn,cur=connect_database()


In [5]:
# CSV file is encoded in latin.
# Quoting flag can be used if the data in csv file is quoted 

players_data=pd.read_csv("Data/players.csv",encoding='latin-1')

teams_data=pd.read_csv("Data/teams.csv",encoding='latin-1',quoting=2)

leagues_data=pd.read_csv("Data/leagues.csv",quoting=2)

games_data=pd.read_csv("Data/games.csv",encoding='latin-1')

shots_data=pd.read_csv("Data/shots.csv",encoding='latin-1')

team_stats=pd.read_csv("Data/teamstats.csv",encoding='latin-1')

player_stats_data=pd.read_csv("Data/playerstats.csv",encoding='latin-1')

In [22]:
# Cleaning the data 
# Only selecting the columns that required for loading into table 
games_data_clean=games_data[["gameID","leagueID","season","date","homeTeamID","awayTeamID","homeGoals","awayGoals","homeProbability","drawProbability"]]

team_stats_clean=team_stats[["gameID","teamID","season","date","location","goals","xGoals","shots","shotsOnTarget","deep"]]

player_stats_data_clean=player_stats_data[["gameID","playerID","goals","ownGoals","shots","xGoals","xGoalsChain","assists","keyPasses"]]

In [7]:
# Validating whether data is in data frame
players_data.head()

,playerID,name
0,560,Sergio Romero
1,557,Matteo Darmian
2,548,Daley Blind
3,628,Chris Smalling
4,1006,Luke Shaw


In [8]:
teams_data.head()

,teamID,name
0,71.0,Aston Villa
1,72.0,Everton
2,74.0,Southampton
3,75.0,Leicester
4,76.0,West Bromwich Albion


In [9]:
leagues_data.head()

,leagueID,name,understatNotation
0,1.0,Premier League,EPL
1,2.0,Serie A,Serie_A
2,3.0,Bundesliga,Bundesliga
3,4.0,La Liga,La_liga
4,5.0,Ligue 1,Ligue_1


In [10]:
games_data_clean.head()

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.2843,0.3999
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.3574,0.3500
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.2988,0.4337
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.6422,0.2057
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.1461,0.2159


In [11]:
shots_data.head()

,gameID,shooterID,assisterID,minute,situation,lastAction,shotType,shotResult,xGoal,positionX,positionY
0,81,554,0,27,DirectFreekick,Standard,LeftFoot,BlockedShot,0.104347,0.794,0.421
1,81,555,631,27,SetPiece,Pass,RightFoot,BlockedShot,0.064342,0.860,0.627
2,81,554,629,35,OpenPlay,Pass,LeftFoot,BlockedShot,0.057157,0.843,0.333
3,81,554,0,35,OpenPlay,Tackle,LeftFoot,MissedShots,0.092141,0.848,0.533
4,81,555,654,40,OpenPlay,BallRecovery,RightFoot,BlockedShot,0.035742,0.812,0.707


In [12]:
team_stats_clean.head()

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep
0,81,89,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4
1,81,82,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10
2,82,73,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11
3,82,71,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2
4,83,72,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5


In [13]:
player_stats_data.head()

,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,position,positionOrder,yellowCard,redCard,time,substituteIn,substituteOut,leagueID
0,81,560,0,0,0,0.0,0.000000,0.000000,0,0,0.000000,GK,1,0,0,90,0,0,1
1,81,557,0,0,0,0.0,0.106513,0.106513,0,1,0.106513,DR,2,0,0,82,222605,0,1
2,81,548,0,0,0,0.0,0.127738,0.127738,0,0,0.000000,DC,3,0,0,90,0,0,1
3,81,628,0,0,0,0.0,0.106513,0.106513,0,0,0.000000,DC,3,0,0,90,0,0,1
4,81,1006,0,0,0,0.0,0.021225,0.021225,0,0,0.000000,DL,4,0,0,90,0,0,1


In [ ]:
#insert data into players table from dataframe 
insert_data(conn,cur,players_data,players_insert_query)

In [ ]:
insert_data(conn,cur,teams_data,teams_insert_query)

In [ ]:
insert_data(conn,cur,leagues_data,leagues_insert_query)

In [ ]:
games_data_clean=games_data_clean

In [ ]:
for i,row in games_data.iterrows():
    try:
        cur.execute(games_insert_query,(row.gameID,row.leagueID,row.season,pd.to_datetime(row["date"],format='%Y-%m-%d %H:%M:%S'),row.homeTeamID,
                row.awayTeamID,row.homeGoals,row.awayGoals,row.homeProbability,row.drawProbability))
        conn.commit()
    except psycopg2.Error as e:
        print(e)
        conn.rollback()

In [ ]:
insert_data(conn,cur,shots_data,shots_insert_query)

In [ ]:
for i,row in team_stats_clean.iterrows():
    try:
        cur.execute(team_stats_insert_query,((row.gameID,row.teamID,row.season,pd.to_datetime(row["date"],format='%Y-%m-%d %H:%M:%S'),row.location,
                row.goals,row.xGoals,row.shots,row.shotsOnTarget,row.deep)))
        conn.commit()
    except psycopg2.Error as e:
        print(e)
        conn.rollback()

In [ ]:
insert_data(conn,cur,player_stats_data_clean,player_stats_insert_query)

In [16]:
# Post validation after inserting the records
cur.execute(players_select);
for row in cur:
    print(list(row))

[560, 'Sergio Romero']
[557, 'Matteo Darmian']
[548, 'Daley Blind']
[628, 'Chris Smalling']
[1006, 'Luke Shaw']


In [ ]:
cur.execute(teams_select);
for row in cur:
    print(list(row))

In [ ]:
cur.execute(leagues_select);
for row in cur:
    print(list(row))

In [ ]:
cur.execute(games_select);
for row in cur:
    print(list(row))

In [ ]:
cur.execute(shots_select);
for row in cur:
    print(list(row))

In [ ]:
cur.execute(team_stats_select);
for row in cur:
    print(list(row))

In [ ]:
cur.execute(players_stats_select);
for row in cur:
    print(list(row))

In [ ]:
#Fetch the top 5 players in terms of goals
cur.execute(top_players);

In [ ]:
print('player_id','player_name','total_goals')
for row in cur:
    print(list(row))

In [ ]:
conn.close()